In [1]:
import re

In [2]:
questions = [
    {
        'id': 0,
        'question': 'What is your name?',
        'attribute': 'name',
        'type': 'text',
        'conditions': None  # Always shown first
    },
    {
        'id': 1,
        'question': 'Which gender perfume are you looking for?',
        'type': 'multi-choice',
        'attribute': 'gender',
        'answers': {1: 'Women', 2: 'Men', 3: 'Unisex'},
        'conditions': None  
    },
    {
        'id': 2,
        'question': 'Are you shopping for yourself or buying a gift?',
        'type': 'single-choice',
        'attribute': 'target',
        'answers': {1: 'Myself', 2: 'Gift'},
        'conditions': None  
    },
    {
        'id': 3,
        'question': 'Which personality best describes the recipient?',
        'type': 'single-choice',
        'attribute': 'personality',
        'answers': {1: 'Adventurous / Bold', 2: 'Romantic / Elegant', 3: 'Fresh / Energetic', 4: 'Warm / Comforting', 5: 'Exotic / Mysterious'},
        'conditions': {'target': [2]}  
    },
    
    # Personality-specific questions
    {
        'id': 4,
        'question': 'What kind of bold scent would they prefer?',
        'type': 'multi-choice',
        'attribute': 'bold-scent',
        'answers': {1: 'Woody (strong, earthy)', 2: 'Spicy (warm, intense)', 3: 'Oriental (rich, exotic)'},
        'conditions': {'personality': [1]}  # Only if Adventurous/Bold
    },
    {
        'id': 5,
        'question': 'Which elegant note fits their style?',
        'type': 'multi-choice',
        'attribute': 'elegant-scent',
        'answers': {1: 'Floral and Oriental (classic, feminine)', 2: 'Sweet Aromatic (soft, charming)'},
        'conditions': {'personality': [2]}  # Only if Romantic/Elegant
    },
    {
        'id': 6,
        'question': 'Which fresh vibe suits them best?',
        'type': 'multi-choice',
        'attribute': 'fresh-scent',
        'answers': {1: 'Fresh (clean, crisp)', 2: 'Citrus (zesty, bright)', 3: 'Fruity (sweet, lively)'},
        'conditions': {'personality': [3]}  # Only if Fresh/Energetic
    },
    {
        'id': 7,
        'question': 'Which comforting scent would they enjoy?',
        'type': 'multi-choice',
        'attribute': 'comforting-scent',
        'answers': {1: 'Vanilla (sweet, cozy)', 2: 'Musk (soft, sensual)', 3: 'Sandalwood (smooth, woody)'},
        'conditions': {'personality': [4]}  # Only if Warm/Comforting
    },
    {
        'id': 8,
        'question': 'Which mysterious scent appeals to them?',
        'type': 'multi-choice',
        'attribute': 'mysterious-scent',
        'answers': {1: 'Woody and Spicy (bold, complex)', 2: 'Arabian (rich, luxurious)'},
        'conditions': {'personality': [5]}  # Only if Exotic/Mysterious
    },
    
    # Myself flow
    {
        'id': 9,
        'question': 'Which season do you prefer the perfume for?',
        'type': 'single-choice',
        'attribute': 'season',
        'answers': {1: 'Spring / Summer', 2: 'Fall / Winter'},
        'conditions': {'target': [1]}  
    },
        
        # Occasion questions
    {
        'id': 10,
        'question': 'What occasion is the perfume for?',
        'type': 'single-choice',
        'attribute': 'occasion',
        'answers': {1: 'Casual', 2: 'Formal', 3: 'Romantic', 4: 'Party', 5: 'Work'},
        'conditions': {'season': [1]} 
    },
    {
        'id': 11,
        'question': 'What occasion is the perfume for?',
        'type': 'single-choice',
        'attribute': 'occasion',
        'answers': {1: 'Casual', 2: 'Formal', 3: 'Romantic', 4: 'Party', 5: 'Work'},
        'conditions': {'season': [2]} 
    },
        # Preferred scents (Myself route)
    {
        'id': 12,
        'question': 'Which fresh vibe suits them best?',
        'type': 'single-choice',
        'attribute': 'fresh-scent-self',
        'answers': {1: 'Fresh (clean, crisp)', 2: 'Citrus (zesty, bright)', 3: 'Fruity (sweet, lively)'},
        'conditions': {'season': [1], 'occasion': [1, 4]}  
    },
    {
        'id': 13,
        'question': 'Which elegant note fits their style?',
        'type': 'multi-choice',
        'attribute': 'elegant-scent-self',
        'answers': {1: 'Floral and Oriental (classic, feminine)', 2: 'Sweet Aromatic (soft, charming)'},
        'conditions': {'season': [1], 'occasion': [2, 3]}  
    },
    {
        'id': 14,
        'question': 'Which clean or aromatic scent do they prefer?',
        'type': 'multi-choice',
        'attribute': 'clean-aromatic-scent-self',
        'answers': {1: 'Clean (fresh, crisp)', 2: 'Aromatic (herbal, lively)', 3: 'Woody (warm, classic)'},
        'conditions': {'season': [1, 2], 'occasion': [5]}  
    },
    {
        'id': 15,
        'question': 'What kind of bold scent do they prefer?',
        'type': 'multi-choice',
        'attribute': 'bold-scent-self',
        'answers': {1: 'Woody (strong, earthy)', 2: 'Spicy (warm, intense)', 3: 'Oriental (rich, exotic)'},
        'conditions': {'season': [2], 'occasion': [1, 4]}  
    },
    {
        'id': 16,
        'question': 'Which comforting scent would they enjoy?',
        'type': 'multi-choice',
        'attribute': 'comforting-scent-self',
        'answers': {1: 'Vanilla (sweet, cozy)', 2: 'Musk (soft, sensual)', 3: 'Sandalwood (smooth, woody)'},
        'conditions': {'season': [2], 'occasion': [2, 3]}
    },
    
    
    # Brand category question
    {
        'id': 17,
        'question': 'Please choose your preferred brand category?',
        'type': 'single-choice',
        'attribute': 'brand-category',
        'answers': {
            1: 'Luxury: (Chanel, Dior, Gucci, etc.)',
            2: 'Mid-Range: (Hugo Boss, Calvin Klein, Marc Jacobs, etc.)',
            3: 'Designer: (Versace, Michael Kors, Coach, etc.)',
            4: 'Niche: (Arabian Oud, Rasasi, Ajmal, etc.)',
            5: 'Affordable: (Avon, Revlon, Elizabeth Arden, etc.)'
        },
        'conditions': None  
    },
    
    # Budget questions (conditional based on brand category)
    {
        'id': 18,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-luxury',  # Changed from 'budget' to 'budget-luxury'
        'answers': {1: '$100 - $200', 2: '$200 - $300', 3: '$300+'},
        'conditions': {'brand-category': [1, 4]}  # For Luxury and Niche
    },
    {
        'id': 19,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-midrange',  # Changed from 'budget' to 'budget-midrange'
        'answers': {1: '$50 - $100', 2: '$100 - $150', 3: '$150+'},
        'conditions': {'brand-category': [2, 3]}  # For Mid-Range and Designer
    },
    {
        'id': 20,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-affordable',  # Changed from 'budget' to 'budget-affordable'
        'answers': {1: 'Under $50', 2: '$50 - $75', 3: '$75+'},
        'conditions': {'brand-category': [5]}  # For Affordable
    } 
]

In [3]:
user_profile = {
    # Basic Information
    "name": "",
    
    # Q1: Gender preferences (multi-choice)
    "gender": [False, False, False],  # [Women, Men, Unisex]
    
    # Q2: Shopping target (single choice)
    "target": [False, False],  # [Myself, Gift]
    
    # Q3: Personality types (single choice) - only for Gift route
    "personality": [False, False, False, False, False],  # [Adventurous/Bold, Romantic/Elegant, Fresh/Energetic, Warm/Comforting, Exotic/Mysterious]
    
    # Q4-8: Personality-specific scent preferences (for Gift route)
    "bold-scent": [False, False, False],  # [Woody, Spicy, Oriental]
    "elegant-scent": [False, False],  # [Floral and Oriental, Sweet Aromatic]
    "fresh-scent": [False, False, False],  # [Fresh, Citrus, Fruity]
    "comforting-scent": [False, False, False],  # [Vanilla, Musk, Sandalwood]
    "mysterious-scent": [False, False],  # [Woody and Spicy, Arabian]
    
    # Q9: Season preference (single choice) - only for Myself route
    "season": [False, False],  # [Spring/Summer, Fall/Winter]
    
    # Q10-11: Occasion (single choice) - conditional on season
    "occasion": [False, False, False, False, False],  # [Casual, Formal, Romantic, Party, Work]
    
    # Q12-16: Scent preferences for Myself route (conditional on season + occasion)
    "fresh-scent-self": [False, False, False],  # [Fresh, Citrus, Fruity] - Spring/Summer + Casual/Party
    "elegant-scent-self": [False, False],  # [Floral and Oriental, Sweet Aromatic] - Spring/Summer + Formal/Romantic
    "clean-aromatic-scent-self": [False, False, False],  # [Clean, Aromatic, Woody] - Any season + Work
    "bold-scent-self": [False, False, False],  # [Woody, Spicy, Oriental] - Fall/Winter + Casual/Party
    "comforting-scent-self": [False, False, False],  # [Vanilla, Musk, Sandalwood] - Fall/Winter + Formal/Romantic
    
    # Q17: Brand category (single choice) - always asked
    "brand-category": [False, False, False, False, False],  # [Luxury, Mid-Range, Designer, Niche, Affordable]
    
    # Q18-20: Budget (single choice) - conditional on brand category
    "budget-luxury": [False, False, False],      # For Luxury/Niche brands
    "budget-midrange": [False, False, False],    # For Mid-Range/Designer brands  
    "budget-affordable": [False, False, False],  # For Affordable brands}
}

In [4]:
q = questions[1]
valid_range= len(q['answers'].keys())
valid_range
    

3

In [ ]:
# Check input from user and display error if user chose invalid option 
def check_input(user_input_list, valid_range): 
    
    if not user_input_list:
        print("Invalid choice (empty).")
        return False
    
    for user_input in user_input_list:
        
        if int(user_input) and 1 <= int(user_input) <= valid_range:
            print(f"{user_input}")
            
        else:
            print("Invalid choice")
            return False
    
    return True

In [6]:
def multi_choice_question(question_id):
    q = questions[question_id]
    valid_range = len(q['answers'])
    
    print(q['question'])
    
    # Save and only print answer options for type "multi-choice" questions
    user_input=''
    print("Options: ")
    for idx, key in q['answers'].items():
        print(f"{idx}. {key}")
                
    # Keep asking until valid
    while True:
        user_input = input(
            f"Input a number from 1 to {valid_range}"
            "(e.g: 1, 2, 3)"
        ).strip()
        
        user_input_list = [x for x in re.split(r'[,\s]+', user_input) if x]
                
        if check_input(user_input_list, valid_range):
            break
        
    # Update boolean flags
    result = [False]*valid_range
    for user_input in user_input_list:
        result[int(user_input)-1] = True
        print(f"You input {q['answers'].get(int(user_input))}")
                
    # Save in user profile
    user_profile[q['attribute']] = result
    
    return user_input_list

In [ ]:
def single_choice_question(question_id):
    q = questions[question_id]
    valid_range = len(q['answers'])
    
    print(q['question'])
    
    # Save and only print answer options for type "uni-choice" questions
    user_input=''
    print("Options: ")
    for idx, key in q['answers'].items():
        print(f"{idx}. {key}")
                
    # Keep asking until valid        
    while True:
        user_input = input(
            f"Select one option from 1 to {valid_range}"
            "(e.g: 1)"
        )

        user_input_list = [user_input]
        
        if check_input(user_input_list, valid_range) and len(user_input) == 1:
            break
        
    # Update boolean flags
    result = [False]*valid_range
    result[int(user_input)-1] = True
    print(f"You input {q['answers'].get(int(user_input))}")

    
    # Save in user profile
    user_profile[q['attribute']] = result
    
    return int(user_input)

In [8]:
def text_question(question_id):
    q = questions[question_id]
    print(q['question'])
    
    if q['type'] == 'text':
        user_input = str(input(f"Your name: "))
        user_profile[q['attribute']] = user_input
        print(user_input)

In [9]:
def ask_question(question_id):
    q = questions[question_id]
    if q['type'] == 'multi-choice':
        return multi_choice_question(question_id)
    elif q['type'] == 'single-choice':
        return single_choice_question(question_id)
    elif q['type'] == 'text':
        return text_question(question_id)

In [10]:
def clean_user_input(user_profile, questions):
    # Create a mapping from attribute names to their answer dictionaries
    attr_to_answers = {}
    for q in questions:
        if 'answers' in q:
            attr_to_answers[q['attribute']] = q['answers']
    
    output = {}
    for attr, value in user_profile.items():
        if isinstance(value, list): # If value is a list
            # Find selected indices (True values)
            chosen_indices = [i+1 for i, v in enumerate(value) if v]
            if chosen_indices:
                # Convert indices to actual string values if mapping exists
                if attr in attr_to_answers:
                    chosen_values = []
                    # Loop through every selected chosen indices
                    for idx in chosen_indices:
                        # Check if that index exist in the answer that we are mapping
                        if idx in attr_to_answers[attr]:
                            # Look up the actual string value of this index
                            string_value = attr_to_answers[attr][idx]   
                            # Add that to our chosen value list
                            chosen_values.append(string_value)
                            
                    # Return single value if only one choice, otherwise return list                    
                    if len(chosen_values) == 1:
                        output[attr] = chosen_values[0]
                    else:
                        output[attr] = chosen_values
                        
                else:
                    output[attr] = chosen_indices
        elif isinstance(value, dict):
            # Find True values in nested dictionaries
            chosen_keys = [k for k, v in value.items() if v]
            if chosen_keys:
                output[attr] = chosen_keys[0] if len(chosen_keys) == 1 else chosen_keys
        else:
            # For strings and other values
            if value != "" and value is not None:
                output[attr] = value
    return output

In [ ]:
def main():
    print("Welcome to the Fragrance Recommendation System!")

    # Q0: Name of user
    ask_question(0)
    
    # Q1: Gender (multi-choice)
    ask_question(1)
    
    # Q2: Target (single choice - myself or gift)
    
    target_choice = ask_question(2)
    
    # Branch based on target choice
    if target_choice == 1: #Myself flow
        print("\n-------- Shopping for yourself ---------\n")   
        
        # Q9: Season 
        season_choice = ask_question(9)
        
        # Q10 or Q11: Occasion (based on season)
        if season_choice == 1: # Spring/Summer
            occasion_choice = ask_question(10)
        else: # Fall/Winter
            occasion_choice = ask_question(11)
            
        # Additional scent questions based on season and occasion
        if season_choice == 1: # Spring/Summer
            if occasion_choice in [1, 4]: # Casual or Party
                ask_question(12) # Fresh scent
            elif occasion_choice in [2, 3]: # Formal or Romantic
                ask_question(13) # Elegant scent
            elif occasion_choice == 5:
                ask_question(14) # Clean/Aromatic
        else: # Fall/Winter
            if occasion_choice in [1, 4]: # Casual or Party
                ask_question(15) # Bold scent
            elif occasion_choice in [2, 3]: # Formal or Romantic
                ask_question(16) # Comforting scent 
            elif occasion_choice == 5: # Work
                ask_question(14)
                
    elif target_choice == 2: #Gift flow
        print("\n-------- Shopping for a gift ---------\n")   
        
        # Q3: Personality
        personality_choice = ask_question(3)
        
        # Personality-specific scent questions
        if personality_choice == 1: # Adventurous/Bold
            ask_question(4)
        elif personality_choice == 2: # Romantic/Elegant
            ask_question(5)
        elif personality_choice == 3:
            ask_question(6)
        elif personality_choice == 4:
            ask_question(7)
        elif personality_choice == 5:
            ask_question(8)
    
    # Mutual questions for both routes
    print("\n-------- Brand and Budget Information --------")
    
    # Q17: Brand category
    brand_choice = ask_question(17)
    
    # Budget question based on brand category
    if brand_choice in [1, 4]: # Luxury or Niche
        ask_question(18)
    elif brand_choice in [2, 3]:
        ask_question(19)
    elif brand_choice == 5: # Affordable
        ask_question(20)
        
    # Clean output showing only selected preferences
    final_profile = clean_user_input(user_profile, questions)
    
    print(f"\nYour Profile Summary: ")
    for key, value in final_profile.items():
        print(f"{key}: {value}")
        
    return final_profile

In [12]:
if __name__ == "__main__":
    main()


Welcome to the Fragrance Recommendation System!
What is your name?
ellie
Which gender perfume are you looking for?
Options: 
1. Women
2. Men
3. Unisex
1
You input Women
Are you shopping for yourself or buying a gift?
Options: 
1. Myself
2. Gift


ValueError: invalid literal for int() with base 10: '1, 2'